# Main function definition
Queue from screen-analisis-ai where :
- status = 1
- jenis-analisa = 10
- order by created desc, limit 1

### Input as proces started
Record header and parameter information

 * Kesepakatan status di kolom screen_analisis_ai.status
 * 1 --> baru diinput
 * 2 --> lagi dikerjakan
 * 3 --> proses berhasil
 * 4 --> proses gagal

### Steps Explanation:
1. Data Loading and Cleaning: Load the JSON data and clean the text data.
2. Feature Extraction: Convert the cleaned text into numerical features using TF-IDF.
3. Optimal Cluster Determination: Use the Elbow Method and Silhouette Score to determine the optimal number of clusters. The optimal number of clusters is determined based on the highest silhouette score.
4. Clustering: Perform clustering using K-Means with the optimal number of clusters.
5. Print Cluster Content: Print the content of each cluster, including the number of members in each cluster.
6. PCA Scatter Plot: Visualize the clusters using a 2D scatter plot with PCA.

This script will dynamically determine the optimal number of clusters, perform clustering, and print the content of each cluster along with the number of members in each cluster.

In [1]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score
import json
import numpy as np
import plotly.express as px
import zipfile
import os
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import numpy as np
from collections import Counter
from scipy.sparse import vstack
from scipy.sparse import csr_matrix
from sklearn.decomposition import PCA

## Internal Data Connector
import data_connector

# Data cleaning function
def clean_text(text):
    text = text.lower()
    text = re.sub(r'@\w+', '', text)  # Remove mentions
    text = re.sub(r'#\w+', '', text)  # Remove hashtags
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'[^a-z\s]', '', text)  # Remove punctuation and numbers
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    return text

def cluster_and_return(df, max_clusters=15, batch_size=10000):
    # Clean the text data
    df['cleaned_text'] = df['text_content'].apply(clean_text)
    
    # Initialize the TF-IDF vectorizer
    vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)
    
    # Fit the vectorizer on the entire dataset to establish the vocabulary
    vectorizer.fit(df['cleaned_text'])
    
    # Transform the data in batches
    n_batches = int(np.ceil(len(df) / batch_size))
    X = csr_matrix((0, len(vectorizer.vocabulary_)), dtype=np.float64)
    
    for i in range(n_batches):
        batch_texts = df['cleaned_text'][i * batch_size:(i + 1) * batch_size]
        X_batch = vectorizer.transform(batch_texts)
        X = vstack([X, X_batch])
    
    # Determine the optimal number of clusters using Elbow Method and Silhouette Score
    def determine_optimal_clusters(X, max_clusters):
        wcss = []
        silhouette_scores = []
        for i in range(2, max_clusters):  # start from 2 clusters
            kmeans = MiniBatchKMeans(n_clusters=i, random_state=42, batch_size=batch_size)
            kmeans.fit(X)
            wcss.append(kmeans.inertia_)
            if i > 1:
                silhouette_avg = silhouette_score(X, kmeans.labels_)
                silhouette_scores.append(silhouette_avg)

        # Determine optimal number of clusters based on highest silhouette score
        optimal_clusters = silhouette_scores.index(max(silhouette_scores)) + 2  # +2 because silhouette_scores starts from 2 clusters
        return optimal_clusters

    optimal_clusters = determine_optimal_clusters(X, max_clusters)
    print(f'Optimal number of clusters: {optimal_clusters}')

    # Perform clustering with the optimal number of clusters
    kmeans = MiniBatchKMeans(n_clusters=optimal_clusters, random_state=42, batch_size=batch_size)
    kmeans.fit(X)
    df['cluster_no'] = kmeans.labels_

    # Calculate average similarity score for each cluster using sparse matrix operations
    cluster_scores = []
    cluster_avg_similarities = []

    for cluster in range(optimal_clusters):
        indices = np.where(kmeans.labels_ == cluster)[0]
        if len(indices) > 1:
            cluster_sim = cosine_similarity(X[indices])
            avg_sim = cluster_sim[np.triu_indices(len(indices), k=1)].mean()
        else:
            avg_sim = 1.0  # If there's only one member in the cluster, similarity is 1
        cluster_scores.extend([avg_sim] * len(indices))
        cluster_avg_similarities.append(avg_sim)

    df['cluster_score'] = cluster_scores

    # calculate the PCA metric
    X = TfidfVectorizer(max_features=1000).fit_transform(df['text_content'])
    pca = PCA(n_components=2, random_state=42)
    pca_components = pca.fit_transform(X.toarray())
    df['pca1'] = pca_components[:, 0]
    df['pca2'] = pca_components[:, 1]

    # Drop the temporary cleaned_text column
    df = df.drop(columns=['cleaned_text'])

    # Return the updated DataFrame and cluster average similarities
    return df, cluster_avg_similarities



# Grab Jobs Queue
ID Jobs ada di screen-analisis-ai, dengan monitoring-id relais ke monitoring_search.id
Diambil untuk jenis_analisa = '10' untuk text clustering

In [2]:
strSQL = """
select 	a.id as jobsid,
        a.*,
		d.id as key_monitoring_media_social, 
		e.id as key_monitoring_media_online,
		c.*
from screen_analisis_ai a 
	inner join monitoring_search c 
		on cast(c.id as varchar) = cast(a.monitoring_id as varchar) 
	inner join monitoring_media_social d
		on d.monitoring_id = c.monitoring_id
	left outer join monitoring_media_online e
		on e.monitoring_id = c.monitoring_id 
where a.jenis_analisa = '10'
and a.status = 1
order by a.created desc 
limit 1
"""

df_job = data_connector.execute_query_psql(strSQL)
if len(df_job) == 0:
    # get out, nothing to do
    print('Zero jobs, quitting now')
    quit()
    
similarity_treshold = 0.9
i_process_id = df_job['jobsid'][0]
screen_name = ''
database_keyword_id = df_job['key_monitoring_media_social'][0]
social_media_monitoring_id = df_job['key_monitoring_media_social'][0]
media_online_monitoring_id = df_job['key_monitoring_media_online'][0]

# print(database_keyword_id)
print(similarity_treshold)
print(i_process_id)
print(social_media_monitoring_id)
print(media_online_monitoring_id)


# Prepare SQL Statement
print(i_process_id)
sql = "update screen_analisis_ai set status = 2, last_status_update = now(), start_process = now() where id = %s"
sql = sql.replace('%s', str(i_process_id))

print(sql)
row_count = data_connector.execute_query_psql(sql)
print('update ' + str(row_count) + ' rows')


0.9
5079
30fe3eee-55b4-489b-95df-21487eb68a05
None
5079
update screen_analisis_ai set status = 2, last_status_update = now(), start_process = now() where id = 5079
update 1 rows


## Processing jobs from queue line
- Check for each social media or media online platform,
- Get the data accordingly
- process to cluster-and-return function
- recording result to database

### Recording Result to database

In [3]:
# prepare function to record result
def record_result(result_df, cluster_avg_similarities, i_platform_id):
    # result_df, cluster_avg_similarities
    for i in range(0, len(result_df)):
        ssql = """
        insert into ret_cluster_result_monitor (ref_id, cluster_no, platform_id,
        job_id, cluster_score,pca_1, pca_2) 
        values ('%s',%s, %s, %s, %s, '%s', '%s')
        """

        # builds str 
        ssql = ssql % (str(result_df['ref_id'][i]), str(result_df['cluster_no'][i]), str(i_platform_id), str(i_process_id), str(cluster_avg_similarities[result_df['cluster_no'][i]]), str(result_df['pca1'][i]), str(result_df['pca2'][i]))
        # print(ssql)
        data_connector.execute_query_psql(ssql)



In [4]:
# Processing jobs for each platform
# 10 = tiktok
# 20 = youtube
# 30 = instagram_post
# 40 = facebook_post
# 50 = google_result

iRowCount = 0

if df_job['is_tiktok'][0]:
    sql = '''
    select id as ref_id, "desc" as text_content 
    from tiktok 
    where monitoring_id = '%s' -- 10
    '''
    i_platform_id = 10
    df = data_connector.execute_query_psql(sql % (social_media_monitoring_id))
    if len(df) != 0:
        result_df, cluster_avg_similarities = cluster_and_return(df)
        record_result(result_df, cluster_avg_similarities, i_platform_id)
        iRowCount = iRowCount + len(df)

if df_job['is_youtube'][0]:
    sql = '''
    select 	id as ref_id, title as text_content
    from youtube 
    where monitoring_id = '%s' -- 20
    '''
    i_platform_id = 20
    df = data_connector.execute_query_psql(sql % (social_media_monitoring_id))
    if len(df) != 0:
        result_df, cluster_avg_similarities = cluster_and_return(df)
        record_result(result_df, cluster_avg_similarities, i_platform_id)
        iRowCount = iRowCount + len(df)

if df_job['is_instagram'][0]:
    sql = '''
    select id as ref_id, content as text_content
    from instagram_post 
    where monitoring_id = '%s' -- 30
    '''
    i_platform_id = 30
    df = data_connector.execute_query_psql(sql % (social_media_monitoring_id))
    if len(df) != 0:
        result_df, cluster_avg_similarities = cluster_and_return(df)
        record_result(result_df, cluster_avg_similarities, i_platform_id)
        iRowCount = iRowCount + len(df)

if df_job['is_facebook'][0]:
    sql = '''
    select id as ref_id, "description" as text_content
    from facebook_post 
    where monitoring_id = '%s' -- 40
    and length(trim("description")) > 0
    '''
    i_platform_id = 40
    df = data_connector.execute_query_psql(sql % (social_media_monitoring_id))
    if len(df) != 0:
        result_df, cluster_avg_similarities = cluster_and_return(df)
        record_result(result_df, cluster_avg_similarities, i_platform_id)
        iRowCount = iRowCount + len(df)

if df_job['is_google'][0]:
    sql = '''
    select 	id as ref_id, "description" as text_content
    from google_result 
    where monitoring_id = '%s' -- 50
    '''
    i_platform_id = 50
    df = data_connector.execute_query_psql(sql % (social_media_monitoring_id))
    if len(df) != 0:
        result_df, cluster_avg_similarities = cluster_and_return(df)
        record_result(result_df, cluster_avg_similarities, i_platform_id)
        iRowCount = iRowCount + len(df)

if df_job['is_twitter'][0]:
    sql = '''
    select id as ref_id, tweet as text_content
    from twitter_tweets 
    where monitoring_id = '%s' -- 60
    '''
    i_platform_id = 60
    df = data_connector.execute_query_psql(sql % (social_media_monitoring_id))
    if len(df) != 0:
        result_df, cluster_avg_similarities = cluster_and_return(df)
        record_result(result_df, cluster_avg_similarities, i_platform_id)
        iRowCount = iRowCount + len(df)

print("Total Processed Data: " + str(iRowCount))

Optimal number of clusters: 14
Total Processed Data: 210


# Mark Jobs queue as done


In [5]:
# Finishing Jobs
# Create Parameter Record
sql = "insert into ret_analysis_parameter (job_id, param_id, param_name, param_value) values (%s, %s, %s, %s)"
# Execute the query
data_connector.execute_query_psql(sql % (i_process_id, 1, "'#Content Processed'",iRowCount))

# Create Tweet Cluster Record
sql = "update ret_analysis_header set datetime_finish = NOW() where job_id = %s"
# Executing query
data_connector.execute_query_psql(sql % (i_process_id) )

sql = "update screen_analisis_ai set status = 3, duration = EXTRACT(EPOCH FROM (now() - start_process)), end_process = NOW() where id = %s"
data_connector.execute_query_psql(sql % (i_process_id))

print('inserting result finished')



inserting result finished


In [6]:
# wait 10 seconds before finished
import time
time.sleep(10)